In [1]:
### 08_wine.csv 파일 불러들이고 출력
import pandas as pd 
import numpy as np
wine = pd.read_csv('./data/08_wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
print(data.shape,target.shape)

(6497, 3) (6497,)


## 교차검증 : train_test_split() 방식 
####  훈련데이터:검증데이터:테스트 데이터 =  6:2:2
#### 보통 많이 사용하는 방식 

In [10]:
# 훈련데이터와 테스트데이터를 8:2로 쪼개기 
from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)


(5197, 3) (5197,)
(1300, 3) (1300,)


In [11]:
### 2. 훈련데이터를 이용해서 검증데이터 생성하기
sub_input, val_input, sub_target,val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42)
print(sub_input.shape,sub_target.shape)
print(val_input.shape,val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


### 결정트리

In [14]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42) # 괄호안에 넣는 값 사람이 지정 : 하이퍼파라미터, 정확도 높이기 위해 찾아내야한다 
dtc.fit(sub_input, sub_target)
print(dtc.score(sub_input, sub_target))
print(dtc.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [15]:
# 과대적합 
# 훈련데이터 정확도 > 테스트 데이터 정확도 
dtc.score(test_input,test_target)

0.8569230769230769

## 교차 검증 시작 

#### - 훈련데이터와 검증데이터를 구분할 필요없이 내부적으로 구분해서 사용
#### - 내부적으로 구분에 사용하는 데이터는 훈련(train)데이터
#### - 데이터 준비는 기존처럼 훈련과 테스트 데이터만 준비
#### - 검증데이터는 별도로 준비안해도 됨

### 교차검증에 사용되는 주요키워드
#### 3-폴드(fold) 교차검증
#### : 훈련데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
#### : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
#### : k-폴드 교차검증 또는 k-겹 교차검증이라고도 함

#### * 보통 5-폴드 교차검증 또는 10-폴드 교차검증을 사용
#### : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능 

In [16]:
### 사용하는 데이터 다시확인
# test 데이터는 마지막에 한번 사용 
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [37]:
### 교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate

# dtc : 결정트리 훈련모델(다른모델을 사용한 경우 해당모델 적어주면된다)
# 두번째값 : 훈련데이터(fold에서 훈련데이터 쪼갤때 사용)
# 세번째값 : 검증데이터(fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(dtc,train_input,train_target)
print(scores)
# 기본값은 5폴드
# test_score : 각 겹의 평균값

## 딕셔너리의 test_score의 평균값이 최종 훈려모델의 평가점수(정확도)

# 최종 훈련모델의 성능
scores['test_score'].mean()

# 강사님 코딩
print('최종: ' ,np.mean(scores['test_score']))

# 다른방법
sum=0
for i in range (0,5,1) :
    sum = scores['test_score'][i]+sum
    
print(sum/5)    

{'fit_time': array([0.01296473, 0.01296544, 0.01296806, 0.01265216, 0.01196742]), 'score_time': array([0.0010004 , 0.00116301, 0.00199318, 0.00099874, 0.0009973 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종:  0.855300214703487
0.855300214703487


In [46]:
### 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - StratifiedKFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold 는 5, 기본 섞지는 않음 
scores = cross_validate(dtc,train_input, train_target, cv=StratifiedKFold())

print(scores)
print('최종 : ',np.mean(scores['test_score']))

{'fit_time': array([0.01343155, 0.0132575 , 0.01296687, 0.01298308, 0.01197052]), 'score_time': array([0.00080609, 0.00070047, 0.00099778, 0.00099444, 0.00101447]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 :  0.855300214703487


In [51]:
# n_splits : fold 갯수 shuffle : 섞기
splitter = StratifiedKFold(n_splits=20, shuffle = True,random_state=42)
scores = cross_validate(dtc,train_input, train_target, cv=splitter)

print(scores)
print('최종 : ',np.mean(scores['test_score']))

{'fit_time': array([0.00800467, 0.00759912, 0.00697494, 0.00691724, 0.00699687,
       0.00632095, 0.00601888, 0.00598407, 0.00504756, 0.        ,
       0.        , 0.01869774, 0.        , 0.        , 0.01784348,
       0.        , 0.        , 0.01666355, 0.        , 0.01015186]), 'score_time': array([0.        , 0.00100374, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ]), 'test_score': array([0.85384615, 0.85384615, 0.88461538, 0.86923077, 0.84230769,
       0.86538462, 0.87307692, 0.85384615, 0.83461538, 0.84615385,
       0.87307692, 0.86923077, 0.85      , 0.86538462, 0.86153846,
       0.86538462, 0.84615385, 0.83783784, 0.8996139 , 0.86100386])}
최종 :  0.8603073953073954


## 하이퍼파라미터 튜닝(AutoML)

### 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
#### - 이런 개념을 AutoML이라고 합니다
#### - 사용패키지 : sklearn.model_selection
#### - 사용 클래스 : GridSearchCV
#### - 그리드 서치 라고도 합니다 

## 결정트리의 max_depth(트리의 깊이) 값 찾기
#### - 그리드서치(GridSearchCV) 객체(모델) 사용


In [56]:
from sklearn.model_selection import GridSearchCV 


# 찾을 하이퍼파라미터는 딕셔너리로
# 실제 파라미터 변수이름 그대로 찾을 범위 지정
params = {'max_depth' : range(5,20,1)}


#객체(모델)생성
# - 첫번째 값: 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼파라미터 값들(딕셔너리로 정의)
# - 세번째 값 : CPU코어 갯수(-1은 모든 코어 사용, 병렬처리)

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

# 훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [58]:
#  그리드서치의 결과
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요 : 훈련모델이 생성되어 반환됨 
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [61]:
# 그리드서치를 통해 생성된 훈련모델을 통해 평가하기 , 훈련데이터 전체 사용 
dtc = gs.best_estimator_
print(dtc.score(train_input,train_target))

0.9003271117952665


In [62]:
print(gs.best_params_)

{'max_depth': 8}


In [65]:
print(gs.cv_results_)

{'mean_fit_time': array([0.00707402, 0.0072783 , 0.0101254 , 0.00762482, 0.00825768,
       0.00990772, 0.0075491 , 0.0073916 , 0.00698133, 0.00767112,
       0.00757985, 0.00738029, 0.00826235, 0.00786171, 0.00718026]), 'std_fit_time': array([0.00115871, 0.00119593, 0.00270525, 0.00079209, 0.00171705,
       0.00453835, 0.00078498, 0.00049153, 0.00063068, 0.0004164 ,
       0.00048788, 0.00048877, 0.00141276, 0.00074612, 0.00074641]), 'mean_score_time': array([0.00094857, 0.00021181, 0.00065942, 0.00090961, 0.00090041,
       0.00080395, 0.00099735, 0.00039296, 0.00059848, 0.00058208,
       0.00099669, 0.00059862, 0.00053148, 0.00090904, 0.00059834]), 'std_score_time': array([2.88041678e-04, 4.23622131e-04, 3.74074037e-04, 1.75858126e-04,
       4.91923955e-04, 4.13257162e-04, 3.81469727e-07, 4.81366920e-04,
       4.88655740e-04, 5.01716937e-04, 1.51240840e-06, 4.88772355e-04,
       4.51424420e-04, 5.02897240e-04, 4.88538707e-04]), 'param_max_depth': masked_array(data=[5, 6, 7, 8, 

In [72]:
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [73]:
### params 값 확인하기
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [74]:
# 1폴드 조회하기
# max_depth=5 ~ max_depth=19 까지 15번 수행 , max_depth=8이 가장 적합한 값 
print(gs.cv_results_['split0_test_score'])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [76]:
### 훈련모델 최종 검증하기 
# 과대적합
print(dtc.score(test_input, test_target))

0.8584615384615385


In [77]:
print(gs.cv_results_['params'])
print(gs.cv_results_['split0_test_score'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]
[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [86]:
table = pd.DataFrame(gs.cv_results_['params'])
table['split_0_test_score'] = pd.DataFrame(gs.cv_results_['split0_test_score'])
table['split_1_test_score'] = pd.DataFrame(gs.cv_results_['split1_test_score'])
table['split_2_test_score'] = pd.DataFrame(gs.cv_results_['split2_test_score'])
table['split_3_test_score'] = pd.DataFrame(gs.cv_results_['split3_test_score'])
table['split_4_test_score'] = pd.DataFrame(gs.cv_results_['split4_test_score'])
table

,max_depth,split_0_test_score,split_1_test_score,split_2_test_score,split_3_test_score,split_4_test_score
0,5,0.847115,0.863462,0.880654,0.836381,0.861405
1,6,0.848077,0.854808,0.873917,0.843118,0.859480
2,7,0.857692,0.851923,0.875842,0.843118,0.847931
3,8,0.852885,0.854808,0.878730,0.846968,0.865255
4,9,0.857692,0.859615,0.871030,0.842156,0.854668
5,10,0.845192,0.847115,0.876805,0.846968,0.852743
6,11,0.858654,0.850962,0.880654,0.852743,0.848893
7,12,0.867308,0.848077,0.882579,0.848893,0.848893
8,13,0.865385,0.853846,0.873917,0.853705,0.839269
9,14,0.868269,0.851923,0.870067,0.858518,0.841193


###  찾을 하이퍼파라미터 여러개 사용하기

In [101]:
# min_impurity_decrease : 결정트리 노드 분리시 불순도 감소 최소량 
# max_depth :  결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
    
### 그리드서치 훈련시키기
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),
                params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [95]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [96]:
dtc = gs.best_estimator_
print(dtc.score(train_input, train_target))
print(dtc.score(test_input, test_target))

0.892053107562055
0.8615384615384616


In [102]:
######## #################수정필요######################333
### 10-폴드 지정하기
splitter = StratifiedKFold(n_splits=7, shuffle = True, random_state=42)
scores = cross_validate(dtc,train_input,train_target)
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)


                
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [104]:
dtc =gs.best_estimator_
print(dtc.score(train_input, train_target))
print(dtc.score(test_input, test_target))


0.892053107562055
0.8615384615384616


In [ ]:
### 랜덤서치 사용시 아래조건 만족해야
# - 하이퍼파라미터값이 수치인 것들만사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래걸리는 경우
#  사용모듈: uniform(실수값) , randint(정수값)
# - 매개변수를 샘플링할때 확률분포객체를 전달 

In [105]:
from scipy.stats import uniform, randint

In [107]:
### 범위설정
rgen = randint(0,10)
rgen

### 범위내에서 임의값 출력하기
print(rgen.rvs(100))

[0 7 2 9 8 8 3 9 9 2 9 8 3 2 9 7 2 2 4 9 3 2 7 3 6 8 2 7 4 3 0 5 0 9 6 4 5
 9 6 7 6 6 1 8 9 2 7 2 3 9 1 9 6 9 1 0 1 9 0 8 3 0 1 8 1 6 1 8 3 2 6 5 1 3
 4 2 0 5 0 7 4 3 9 2 8 8 3 1 5 9 2 2 0 9 2 0 3 3 7 2]


In [108]:
# 실수값 
# 범위설정 
ugen = uniform(0,1)
# 임의의값 50개 조회
print(ugen.rvs(50))

[0.28941879 0.65277119 0.61722651 0.25847319 0.31802927 0.72400651
 0.77828459 0.27752331 0.55444788 0.80523595 0.09929453 0.15402696
 0.99916752 0.16895867 0.23387033 0.98389405 0.26077521 0.83205209
 0.08562233 0.17277534 0.84181564 0.16705942 0.92374897 0.33581112
 0.85841449 0.56388843 0.30141775 0.93185986 0.76602971 0.52901686
 0.18633288 0.82409183 0.28708396 0.17323761 0.03507325 0.03161638
 0.6670464  0.22414703 0.09682671 0.42905464 0.18774719 0.34090094
 0.6095203  0.35112643 0.38831529 0.89704449 0.87883569 0.12291383
 0.33957813 0.52044302]


In [125]:
###########################################수정 필요 #################################################
### 하이퍼파라미터 매개변수 정의하기
params = {'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델사용, 훈련반복 횟수 100회, 코어 전체, 시드 42
# n_iter=100 : 반복횟수 params에서 uniform 값이 임의로 추출해서 훈련하지만 범위가 없어서 적어준다

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100, n_jobs=-1,random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B031CE8160>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002B032626850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002B0325FDFA0>},
                   random_state=42)

In [127]:
print(rs.best_params_)

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}


In [128]:
print(rs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [129]:
print(rs.cv_results_['mean_test_score'])

[0.86203543 0.86203524 0.86453672 0.8639598  0.86434441 0.86184534
 0.86280669 0.86684608 0.86607592 0.86203561 0.85972662 0.8603054
 0.86338306 0.86145813 0.86222792 0.86145739 0.86761605 0.86261272
 0.86184238 0.86396109 0.86453709 0.86646165 0.86184312 0.86665322
 0.86165044 0.86030484 0.86299845 0.86742356 0.86203543 0.86838658
 0.86319186 0.86819334 0.85799659 0.86242023 0.86203561 0.86299752
 0.86261309 0.86049733 0.86857833 0.86299752 0.86800085 0.86203543
 0.8664635  0.86415451 0.86588473 0.86338362 0.8678091  0.86357611
 0.8624206  0.85857315 0.86203543 0.86742374 0.86607629 0.86838639
 0.86107333 0.86030466 0.86203543 0.86376805 0.86396146 0.86954098
 0.86954079 0.86800104 0.86145739 0.86242023 0.85972662 0.86261327
 0.86684626 0.86703913 0.86472958 0.86280503 0.86607814 0.86530632
 0.86800104 0.86203543 0.86088325 0.86261272 0.86299826 0.86242004
 0.86242023 0.86203543 0.86222773 0.86280558 0.8660787  0.85953413
 0.86800104 0.86145739 0.86184293 0.86588491 0.86184293 0.86761

In [130]:
np.max(rs.cv_results_['mean_test_score'])

0.8695409787517583

In [132]:
dtc = gs.best_estimator_
print(dtc.score(test_input,test_target))

0.8615384615384616
